In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
df = pd.read_excel(r'M:/Risk Management/DW/Scorecard/TUX_Scores/MarchPayloads/Output_Files/March__Final__Gen12and13_wHardcuts_decile.V3.xlsx')

In [3]:
# Dict of threshold tuples

scorecard_thresholds = {
    'Scorecard1': [
        (723, 900),   # Tier 1
        (705, 722),   # Tier 2
        (689, 704),   # Tier 3
        (670, 688),   # Tier 4
        # <= 669 = Decline
    ],
    'Scorecard2': [
        (715, 900),   # Tier 6
        (704, 714),   # Tier 7
        # <= 703 = Decline
    ],   
    'Scorecard3': [
        (717, 900),   # Tier 4
        (706, 716),   # Tier 5
        (697, 705),   # Tier 6
        (689, 696),   # Tier 7
        # <= 688 = Decline 
    ],
    'Scorecard4': [
        (755, 900),   # Tier 1
        (736, 754),   # Tier 2
        (721, 735),   # Tier 3
        (704, 720),   # Tier 4
        # <= 703 = Decline
    ] 
}
        

In [4]:
# Define the Tier Assignment Function
def get_tier(scorecard_name, score):
    tiers = scorecard_thresholds.get(scorecard_name)
    if not tiers:
        return 'Decline'  # Unknown scorecard

    for i, (min_val, max_val) in enumerate(tiers):
        if min_val <= score <= max_val:
            return i + 1  # Tiers are 1-based

    return 'Decline'  # Score doesn't match any tier

In [5]:
# Function to assign tiers to the DataFrame
def assign_tiers(df):
    df['Tier'] = df.apply(lambda row: get_tier(row['CreditScorecardName'], row['AvgEQScore60_40']), axis=1)
    return df

In [6]:
# Deduplication Logic for Scorecards 1 and 4
def dedupe_scorecards(df, scorecards_to_dedupe):
    df_sc = df[df['CreditScorecardName'].isin(scorecards_to_dedupe)]
    df_sc_max = df_sc.loc[df_sc.groupby(['AccountId', 'CreditScorecardName'])['AvgEQScore60_40'].idxmax()]
    df_other = df[~df['CreditScorecardName'].isin(scorecards_to_dedupe)]
    return pd.concat([df_other, df_sc_max], ignore_index=True)

In [7]:
# def handle_dual_applicants(df):
#     # Function to handle dual applicants
#     def process_group(group):
#         # Get all tiers (including Decline and numeric tiers)
#         tiers = group['Tier'].tolist()
        
#         # Remove any non-numeric values (like 'Decline') from tiers list
#         numeric_tiers = [tier for tier in tiers if isinstance(tier, int)]

#         if "Decline" in tiers:
#             # If there's a 'Decline' in any tier, we set it to 'Decline'
#             group['Tier'] = 'Decline'
#         else:
#             # Otherwise, we set the best (max) numeric tier
#             if numeric_tiers:
#                 best_tier = max(numeric_tiers)
#                 group['Tier'] = best_tier
#             else:
#                 # If no valid numeric tiers, set to 'Decline'
#                 group['Tier'] = 'Decline'

#         return group

#     # Apply the function to each group of dual applicants (grouped by 'AccountId')
#     df = df.groupby('AccountId', group_keys=False).apply(process_group)

#     # Reset index to remove ambiguity and get AccountId as a column again
#     df = df.reset_index(drop=True)

#     return df


In [8]:
# Attempt to fix above function

def handle_dual_applicants(df):
    def process_group(group):
        # Get all tiers (including Decline and numeric tiers)
        tiers = group['Tier'].tolist()
        
        # Remove any non-numeric values (like 'Decline') from tiers list
        numeric_tiers = [tier for tier in tiers if isinstance(tier, int)]
        
        # If there's a 'Decline' in any tier, check the scenario:
        if "Decline" in tiers:
            # If one applicant is 'Decline', check if other has a valid tier (non-Decline)
            if numeric_tiers:
                group['Tier'] = max(numeric_tiers)  # Assign the best valid tier if there is one
            else:
                group['Tier'] = 'Decline'  # All declined, so set Decline for the group
        else:
            # If no 'Decline', assign the best (max) numeric tier
            if numeric_tiers:
                best_tier = max(numeric_tiers)
                group['Tier'] = best_tier
            else:
                # If no valid numeric tiers, set to 'Decline'
                group['Tier'] = 'Decline'

        return group

    # Apply the function to each group of dual applicants (grouped by 'AccountId')
    df = df.groupby('AccountId', group_keys=False).apply(process_group)
    df = df.reset_index(drop=True)
    return df


In [9]:
# Final Processing for Collapsing Duplicates Based on Account ID
def collapse_duplicates(df):
    dupes = df[df.duplicated('AccountId', keep=False)]
    collapsed = dupes.groupby('AccountId').apply(collapse_tiers).reset_index(drop=True)
    df_final = pd.concat([df[~df['AccountId'].isin(dupes['AccountId'])], collapsed], ignore_index=True)
    return df_final

In [10]:
# Helper function to collapse tiers based on situation (Best/Worst tier for group)
def collapse_tiers(group):
    tiers = group['Tier'].tolist()
    if 'Decline' in tiers:
        final_tier = 'Decline'
    else:
        final_tier = max(tiers)
    
    row = group.iloc[0].copy()
    row['Tier'] = final_tier
    return row

In [11]:
# Run through functions - Assign tiers to each row
df = assign_tiers(df)

In [12]:
# # Deduplicate specific scorecards by max score
# scorecards_to_dedupe = ['Scorecard1', 'Scorecard4']
# df_clean = dedupe_scorecards(df, scorecards_to_dedupe)


In [13]:
# # Collapse remaining duplicate account entries into final version
# df_final = collapse_duplicates(df_clean)

In [14]:
# Deduplicate Scorecard 1 and Scorecard 4 (this step handles the max score logic)
sc_to_dedup = ['Scorecard1', 'Scorecard4']
df_combined = dedupe_scorecards(df, sc_to_dedup)


In [15]:
# Handle dual applicants: best/worst tier assignments based on situations
df_combined = handle_dual_applicants(df_combined)

C:\Users\dwolters\AppData\Local\Temp\ipykernel_21264\1859703856.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('AccountId', group_keys=False).apply(process_group)


In [16]:
# Collapse duplicates based on accountid and finalize the tier assignments
df_final = collapse_duplicates(df_combined)

C:\Users\dwolters\AppData\Local\Temp\ipykernel_21264\3909711267.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  collapsed = dupes.groupby('AccountId').apply(collapse_tiers).reset_index(drop=True)


In [17]:
# check results
df_final.shape

(16081, 58)

In [18]:
df_final.head(20)

,AccountId,RequestDate,PfsModelName,CreditScorecardVersion,CreditScorecardName,CreditLogScore,CreditAdjustedLogScore,CreditProbablityToDefault,CreditEqualizedScore,CapacityScorecardVersion,...,BK_Declines,row_count,TotalAppIncome,ScorecardName_primary,ScorecardName_coapplicant,EQScore_Quantile,EQScore_Decile,EQScore_Decile10,Final_Scorecard_Segment,Tier
0,8552019,2025-03-05 19:02:25,PRESTIGE-GEN-XII,v1,Scorecard3,1.969187,1.768517,0.545,718,v1,...,0,1,4012.59,NaN,NaN,Q5 (80-100%),Q10 (90-100%),D10 (90-100%),TUX_ThickDirty,1
1,8569614,2025-03-10 17:28:15,PRESTIGE-GEN-XII,v1,Scorecard3,0.726715,0.526044,0.368,647,v1,...,0,1,NaN,NaN,NaN,Q3 (40-60%),Q5 (40-50%),D5 (40-50%),TUX_ThickDirty,Decline
2,8571390,2025-03-17 13:55:41,PRESTIGE-GEN-XII,v1,Scorecard3,0.966148,0.765477,0.402,661,v1,...,0,1,7257.74,NaN,NaN,Q1 (0-20%),Q1 (0-10%),D1 (0-10%),TUX_ThickDirty,Decline
3,8582827,2025-03-20 16:20:44,PRESTIGE-GEN-XII,v1,Scorecard3,0.857709,0.657038,0.388,655,v1,...,0,1,4166.67,NaN,NaN,Q5 (80-100%),Q10 (90-100%),D10 (90-100%),TUX_ThickDirty,1
4,8587968,2025-03-07 22:09:57,PRESTIGE-GEN-XII,v1,Scorecard3,0.347259,0.146588,0.312,625,v1,...,0,1,5903.02,NaN,NaN,Q1 (0-20%),Q2 (10-20%),D2 (10-20%),TUX_ThickDirty,Decline
5,8591822,2025-03-13 19:50:19,PRESTIGE-GEN-XII,v1,Scorecard3,1.075143,0.874472,0.418,667,v1,...,0,1,15700.75,NaN,NaN,Q2 (20-40%),Q3 (20-30%),D3 (20-30%),TUX_ThickDirty,Decline
6,8597179,2025-03-11 21:07:17,PRESTIGE-GEN-XII,v1,Scorecard3,-0.716459,-0.917130,0.162,565,v1,...,0,1,6908.10,NaN,NaN,Q1 (0-20%),Q1 (0-10%),D1 (0-10%),TUX_ThickDirty,Decline
7,8599314,2025-03-27 21:08:17,PRESTIGE-GEN-XII,v1,Scorecard2,0.207301,0.006630,0.298,619,v1,...,0,1,4000.00,NaN,NaN,Q2 (20-40%),Q4 (30-40%),D4 (30-40%),TUX_Thin,Decline
8,8605501,2025-03-19 14:44:34,PRESTIGE-GEN-XII,v1,Scorecard3,0.821025,0.620355,0.382,653,v1,...,0,1,6750.00,NaN,NaN,Q3 (40-60%),Q5 (40-50%),D5 (40-50%),TUX_ThickDirty,Decline
9,8614084,2025-03-18 15:18:19,PRESTIGE-GEN-XII,v1,Scorecard3,1.009371,0.808700,0.408,663,v1,...,0,1,13061.42,NaN,NaN,Q3 (40-60%),Q5 (40-50%),D5 (40-50%),TUX_ThickDirty,Decline


In [19]:
df_final.to_excel(r'M:/Risk Management/DW/Scorecard/TUX_Scores/MarchPayloads/Output_Files/March_TierAssigments_v2.xlsx', index=False)